In [2]:
# **  Import necessary modules**
from classiq import *
from classiq.qmod.symbolic import logical_or
from classiq.execution import ExecutionPreferences

#  Define constants*
size = 4  # Number of qubits to represent the vertices (log2 of 16 nodes)
num_nodes = 2**size  # Total number of nodes (16 nodes)

# **  Function to prepare a qubit in the |-> state**
@qfunc
def prepare_minus(x: QBit):
    
    X(x)
    H(x)

# **  Implement a diffuser oracle for Grover's algorithm**
@qfunc
def diffuzer_oracle(aux: Output[QNum], x: QNum):
    
    aux ^= (x != 0)

# **  Implement the zero diffuser for the quantum walk**
@qfunc
def zero_diffuzer(x: QNum):
   
    aux = QNum('aux')
    allocate(1, aux)
    within_apply(compute=lambda: prepare_minus(aux),
                 action=lambda: diffuzer_oracle)

# ** Define the coin operation for the quantum walk**
def C_iteration(i: int, vertices: QNum, adjacent_vertices: QNum):
   
    prob = [0]*num_nodes  # Initialize probability vector for 16 nodes
    if i == 0:
        prob[i + 1] = 1.0  # If at the first node, move to the right node (node 1)
    elif i == num_nodes - 1:
        prob[i - 1] = 1.0  # If at the last node, move to the left node (node 14)
    else:
        prob[i - 1] = 0.5  # Probability of moving to the left node
        prob[i + 1] = 0.5  # Probability of moving to the right node
    print(f'Node={i}, prob vec ={prob}')
    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))

# ** Apply the coin operator to all vertices**
@qfunc
def C_operator(vertices: QNum, adjacent_vertices: QNum):
       for i in range(num_nodes):
        C_iteration(i, vertices, adjacent_vertices)

# ** Main function to perform the quantum random walk**
@qfunc
def main(vertices: Output[QNum], adjacent_vertices: Output[QNum]):
   
    allocate(size, vertices)  # 1 Allocate qubits for vertices
    hadamard_transform(vertices)  # 2 Apply Hadamard transform to initialize superposition
    allocate(size, adjacent_vertices)  # 3 Allocate qubits for adjacent vertices
    
    C_operator(vertices, adjacent_vertices)  # 4 Apply coin operator
    # S_operator(vertices, adjacent_vertices)  # 5 Apply shift operator ( implemented)

@qfunc
def S_operator(vertices: QNum, adjacent_vertices: QNum):
   
    # Implement the shift operation based on the coin operator's output
    # Update the walker's position accordingly
    pass

# ** Create and synthesize the quantum model**
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Node=0, prob vec =[0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
Node=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
Node=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
Node=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
Node=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
Node=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
Node=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
Node=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0]
Node

ClassiqAPIError: Call to API failed with code 401: Not authenticated
If you need further assistance, please reach out on our Community Slack channel at: https://short.classiq.io/join-slack

In [5]:
!pip install classiq


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
